In [2]:
import numpy as np
A = [[.7, .3], [.4, .6]]
B = [[.1, .4, .5], [.7, .2, .1]]
pi = [.6, .4]
s = [0, 0, 1, 1]    # these are the xs

# Problem 3

### Part A

In [2]:
probs1 = {}

for i in (0, 1, 2):    # these are the observations i,j,k,l
    for j in (0, 1, 2):
        for k in (0, 1, 2):
            for l in (0, 1, 2):
                for a in (0, 1):    # These are the states a,b,c,d
                    for b in (0, 1):
                        for c in (0, 1):
                            for d in (0, 1):
                                state = (a,b,c,d)
                                ob = (i,j,k,l)
                                prob = pi[a] * B[a][i] * A[a][b] * B[b][j] * A[b][c] * B[c][k] * A[c][d] * B[d][l]
                                try: probs1[ob] += prob
                                except:
                                    probs1[ob] = prob

In [7]:
sum(probs1.values())

0.9999999999999994

### Part B

In [4]:
probs2 = {}
for i in (0, 1, 2):    # these are the observations i,j,k,l
    for j in (0, 1, 2):
        for k in (0, 1, 2):
            for l in (0, 1, 2):
                # For each possible observation:
                ob = (i,j,k,l)
                
                a00 = pi[0] * B[0][i]
                a01 = pi[1] * B[1][i]
                
                alpha = [[a00],[a01]]
                
                # Forward pass
                for t in (1,2,3):
                    #                 j is 0                      j is 1
                    at_0 = (alpha[0][t-1] * A[0][0] + alpha[1][t-1] * A[1][0]) * B[0][ob[t]] # i is 0
                    at_1 = (alpha[0][t-1] * A[0][1] + alpha[1][t-1] * A[1][1]) * B[1][ob[t]] # i is 1
                    
                    alpha[0].append(at_0)
                    alpha[1].append(at_1)
                    
                prob = alpha[0][-1] + alpha[1][-1]
                probs2[ob] = prob

In [5]:
sum(probs2.values())

0.9999999999999996

In [1]:
# Check they are the same
for i in (0, 1, 2):
    for j in (0, 1, 2):
        for k in (0, 1, 2):
            for l in (0, 1, 2):
                # For each observation:
                ob = (i,j,k,l)
                print probs1[ob] - probs2[ob] < 1e-10
                

# The HMM class

In [361]:
class hmm(object):
    def __init__(self):#(self, A, B, pi):
#         self.A = A
#         self.B = B
#         self.pi = pi
#         self.num_state = A.shape[0]
        pass
        
    def forward_pass(self, obs):
        alpha = np.zeros((len(obs), self.num_state))
        c1 = 1. / sum(self.pi * self.B[obs[0],:])
        alpha[0,:] = c1 * (self.pi * self.B[obs[0],:])
        cs = [c1]
        
        for t in range(1, len(obs)):
            ct = 1. / sum(self.A.dot(alpha[t-1,:]) * B[obs[t],:])
            alpha[t,:] = ct * (self.A.dot(alpha[t-1,:]) * B[obs[t],:])
            cs.append(ct)
    
        return alpha, np.array(cs)
    
    def backward_pass(self, obs):
        a, c = self.forward_pass(obs)
        T = len(obs) - 1
        beta = np.zeros((len(obs), self.num_state))
        beta[T,:] = np.array([c[T] for _ in range(self.num_state)])
        for t in range(T-1, -1, -1):
            beta[t,:] = c[t] * A.T.dot(B[obs[t+1],:] * beta[t+1])
            
        return beta
    
    def _delta(self, obs, alpha, beta):
        T = len(obs)
        delta = np.zeros((T - 1, self.num_state, self.num_state))
        gamma = np.zeros((len(obs), self.num_state))
        
        for t in range(T-1):
            denom = 0
            for k in range(self.num_state):
                for l in range(self.num_state):
                    denom += alpha[t,k] * self.A[l,k] * self.B[obs[t+1],l] * beta[t+1,l]
            
            for i in range(self.num_state):
                for j in range(self.num_state):
                    delta[t,i,j] = (alpha[t,i]*self.A[j,i]*self.B[obs[t+1],j]*beta[t+1,j]) / denom
                gamma[t,i] = sum(delta[t,i,:])
        
        gamma[T-1,:] = alpha[T-1,:] * beta[T-1,:] / sum(alpha[T-1,:] * beta[T-1,:])
                    
        return delta, gamma
            
        
    def _estimate(self, obs, delta, gamma):
        """
        Estimate better parameter values.
        Parameters
        ----------
        obs : ndarray of shape (T,)
        The observation sequence
        delta : ndarray of shape (T-1,N,N)
        The delta probability array
        gamma : ndarray of shape (T,N)
        The gamma probability array
        """
        self.pi = gamma[0,:]
    
        # update A
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]):
                num = 0
                den = 0
                for t in range(len(obs) - 1):
                    num += delta[t,j,i]
                    den += gamma[t,j]
                A[i,j] = num / den
    
        #update B
        for i in range(self.B.shape[0]):
            for j in range(self.B.shape[1]):
                self.B[i,j] = np.sum(gamma[:,j]*(np.array(obs)==i)) / np.sum(gamma[:,j])
                
        # print self.pi, self.A, self.B

    def fit(self, obs, A, B, pi, max_iter=200, tol=1e-3):
        
        M = len(set(obs))
        N = A.shape[0]
        self.num_state = N
        self.A = A
        self.B = B
        self.pi = pi
        
        p = np.inf
        for i in range(max_iter):
#             print i
            alpha, cs = self.forward_pass(obs)
            P = -(np.log(cs)).sum()
            if (abs(P - p) < tol): break
#             if P < -21550: break
                
            p = P
            beta = self.backward_pass(obs)
            delta, gamma = self._delta(obs, alpha, beta)
            self._estimate(obs, delta, gamma)
            
        
        
    def dp(self,obs):
        ### update for transposed A, B
        delta = np.zeros((len(obs), self.num_state))
        delta[0,:] = self.pi * self.B[:, obs[0]]

        best_path = []
        
        for t in range(1, len(obs)):
            probs = delta[t-1, :].reshape(2, 1) * self.A * self.B[:,obs[t]]

            delta[t,:] = np.max(probs, axis=0)
            i = np.argmax(delta[t,:])
            best = np.argmax(probs[:,i])

            best_path.append(best)

        best_path.append(np.argmax(delta[-1,:]))
        return best_path, np.max(delta[-1,:])
        

In [363]:

import string
import codecs
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)
def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower() #and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    #make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    #convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence


## Test on Declaration

### N = 2

In [364]:
symbols, obs = prep_data('declaration.txt')
M = len(symbols)
N = 2
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = hmm()
h.fit(obs, A, B, pi)
for i in xrange(len(h.B)):
    print "{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1])
    

 , 0.2996, 0.0491
a, 0.1316, 0.0000
b, 0.0000, 0.0226
c, 0.0000, 0.0438
d, 0.0000, 0.0600
e, 0.2370, 0.0000
f, 0.0000, 0.0428
g, 0.0000, 0.0309
h, 0.0005, 0.0827
i, 0.1239, 0.0000
j, 0.0000, 0.0038
k, 0.0004, 0.0030
l, 0.0000, 0.0543
m, 0.0000, 0.0343
n, 0.0000, 0.1150
o, 0.1378, 0.0032
p, 0.0000, 0.0328
q, 0.0000, 0.0014
r, 0.0000, 0.1012
s, 0.0000, 0.1138
t, 0.0000, 0.1523
u, 0.0577, 0.0000
v, 0.0000, 0.0176
w, 0.0000, 0.0231
x, 0.0000, 0.0021
y, 0.0116, 0.0093
z, 0.0000, 0.0010


### N = 3

In [365]:
N = 3
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = hmm()
h.fit(obs, A, B, pi)
for i in xrange(len(h.B)):
    print "{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2])
    

 , 0.2764, 0.0219, 0.1688
a, 0.0000, 0.2006, 0.0000
b, 0.0171, 0.0000, 0.0178
c, 0.0293, 0.0000, 0.0392
d, 0.0515, 0.0000, 0.0399
e, 0.0336, 0.3192, 0.0000
f, 0.0235, 0.0000, 0.0445
g, 0.0281, 0.0000, 0.0188
h, 0.1172, 0.0000, 0.0000
i, 0.0306, 0.1505, 0.0000
j, 0.0054, 0.0000, 0.0000
k, 0.0029, 0.0010, 0.0012
l, 0.0427, 0.0000, 0.0408
m, 0.0191, 0.0000, 0.0352
n, 0.0052, 0.0000, 0.1893
o, 0.0000, 0.2157, 0.0000
p, 0.0291, 0.0000, 0.0207
q, 0.0000, 0.0015, 0.0010
r, 0.0474, 0.0000, 0.1149
s, 0.0631, 0.0000, 0.1174
t, 0.1488, 0.0000, 0.0797
u, 0.0036, 0.0632, 0.0194
v, 0.0038, 0.0000, 0.0253
w, 0.0198, 0.0000, 0.0154
x, 0.0000, 0.0000, 0.0036
y, 0.0016, 0.0264, 0.0055
z, 0.0000, 0.0000, 0.0016


### N=4

In [367]:
N = 4
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = hmm()
h.fit(obs, A, B, pi)
for i in xrange(len(h.B)):
    print "{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2], h.B[i,3])

 , 0.9180, 0.0000, 0.0000, 0.0000
a, 0.0084, 0.1929, 0.0000, 0.0000
b, 0.0000, 0.0130, 0.0354, 0.0000
c, 0.0123, 0.0000, 0.0726, 0.0164
d, 0.0000, 0.0000, 0.0348, 0.0857
e, 0.0000, 0.2212, 0.0000, 0.1473
f, 0.0000, 0.0000, 0.0461, 0.0440
g, 0.0000, 0.0000, 0.0300, 0.0344
h, 0.0000, 0.1067, 0.0510, 0.0000
i, 0.0437, 0.1607, 0.0000, 0.0000
j, 0.0000, 0.0000, 0.0089, 0.0000
k, 0.0002, 0.0015, 0.0056, 0.0000
l, 0.0000, 0.0118, 0.0569, 0.0441
m, 0.0008, 0.0000, 0.0429, 0.0298
n, 0.0000, 0.0000, 0.0183, 0.2037
o, 0.0000, 0.2006, 0.0163, 0.0000
p, 0.0023, 0.0047, 0.0687, 0.0000
q, 0.0000, 0.0000, 0.0033, 0.0000
r, 0.0000, 0.0000, 0.0986, 0.1122
s, 0.0062, 0.0000, 0.0706, 0.1550
t, 0.0000, 0.0000, 0.2551, 0.0822
u, 0.0082, 0.0819, 0.0000, 0.0000
v, 0.0000, 0.0000, 0.0376, 0.0029
w, 0.0000, 0.0000, 0.0402, 0.0112
x, 0.0000, 0.0000, 0.0050, 0.0000
y, 0.0000, 0.0051, 0.0000, 0.0311
z, 0.0000, 0.0000, 0.0022, 0.0000


In these examples, we see that the vowels are pretty consistently grouped together, with a little bit of extra spread as N increases.  The consonants get scattered into different groups in ways that are hard to interpret from a superficial perusing of the results.  Could it be by phoneme?  By frequency?  By what letters most commonly follow or precede them?  Deeper research could perhaps elucidate this mystery, but for an assignment on conference weekend this sufficeth me.

## Test on War and Peace


### N = 2

In [375]:
symbols, obs = prep_data('warandpeace.txt')
M = len(symbols)

In [370]:
N = 2
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = hmm()
h.fit(obs, A, B, pi)
for i in xrange(len(h.B)):
    print u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1])

 , 0.2148, 0.0881
́, 0.0001, 0.0000
а, 0.0000, 0.1751
б, 0.0251, 0.0000
в, 0.0657, 0.0000
г, 0.0297, 0.0000
д, 0.0386, 0.0001
е, 0.0177, 0.1425
ж, 0.0140, 0.0000
з, 0.0253, 0.0000
и, 0.0003, 0.1328
й, 0.0150, 0.0000
к, 0.0498, 0.0011
л, 0.0721, 0.0000
м, 0.0383, 0.0000
н, 0.0976, 0.0000
о, 0.0000, 0.2396
п, 0.0352, 0.0055
р, 0.0599, 0.0000
с, 0.0514, 0.0280
т, 0.0782, 0.0000
у, 0.0000, 0.0587
ф, 0.0018, 0.0003
х, 0.0111, 0.0000
ц, 0.0050, 0.0000
ч, 0.0168, 0.0038
ш, 0.0109, 0.0000
щ, 0.0047, 0.0000
ъ, 0.0003, 0.0003
ы, 0.0000, 0.0374
ь, 0.0000, 0.0445
э, 0.0000, 0.0065
ю, 0.0079, 0.0025
я, 0.0126, 0.0330
ё, 0.0000, 0.0001


### N = 3

In [374]:
N = 3
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = hmm()
h.fit(obs, A, B, pi)
for i in xrange(len(h.B)):
    print u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2])

 , 0.0387, 0.0991, 0.3992
́, 0.0000, 0.0000, 0.0003
а, 0.2079, 0.0000, 0.0000
б, 0.0000, 0.0295, 0.0153
в, 0.0000, 0.0743, 0.0501
г, 0.0000, 0.0333, 0.0127
д, 0.0000, 0.0430, 0.0215
е, 0.1551, 0.0000, 0.0448
ж, 0.0000, 0.0165, 0.0068
з, 0.0000, 0.0285, 0.0183
и, 0.1385, 0.0000, 0.0094
й, 0.0000, 0.0000, 0.0328
к, 0.0000, 0.0643, 0.0322
л, 0.0000, 0.1041, 0.0288
м, 0.0000, 0.0440, 0.0239
н, 0.0000, 0.1303, 0.0432
о, 0.2677, 0.0000, 0.0000
п, 0.0000, 0.0461, 0.0293
р, 0.0000, 0.0881, 0.0132
с, 0.0005, 0.0415, 0.0982
т, 0.0000, 0.0998, 0.0319
у, 0.0640, 0.0000, 0.0029
ф, 0.0000, 0.0010, 0.0019
х, 0.0000, 0.0094, 0.0072
ц, 0.0000, 0.0098, 0.0001
ч, 0.0000, 0.0159, 0.0140
ш, 0.0000, 0.0128, 0.0051
щ, 0.0000, 0.0083, 0.0000
ъ, 0.0006, 0.0002, 0.0000
ы, 0.0380, 0.0000, 0.0000
ь, 0.0483, 0.0000, 0.0000
э, 0.0000, 0.0000, 0.0089
ю, 0.0035, 0.0000, 0.0195
я, 0.0372, 0.0000, 0.0287
ё, 0.0001, 0.0000, 0.0000
﻿, 0.0000, 0.0001, 0.0000


From these we see that the HMM does a pretty good job of grouping the letters up, although since I don't know Russian I can't say for sure whether they are grouped as vowels and consonants or not.  It looks like the letters similar to an a, e, backwards N, o, y, b, and backwards R could function as vowels, and perhaps a couple others.

# Gaussian HMM

In [71]:
# Create some data to train on

n = 2
t = 5000
pi=  [.3, .7]
a = np.array([[.4, .6],[.9, .1]])
mu1 = np.array([2,0])
mu2 = np.array([0,2])
sig1 = np.eye(2)
sig2 = np.array([[.5, 0],[0, 1]])

In [72]:
np.random.seed(117)
def generate_obs(pi, A, mus, sigs, T):
    # According to instructions on Jarvis's blackboard
    x0 = np.random.choice(range(len(pi)), p=pi)
    xs = [x0]
    zs = [np.random.multivariate_normal(mus[x0], sigs[x0])]
    for t in range(T-1):
        newx = np.random.choice(range(len(pi)), p=A[xs[-1]])
        xs.append(newx)
        zs.append(np.random.multivariate_normal(mus[newx], sigs[newx]))
    
    return zs

In [67]:
zs = generate_obs(pi, a, [mu1, mu2], [sig1, sig2], t)

In [68]:
zs[:5]

[array([-0.53381675,  0.98784833]),
 array([ 2.10233039,  0.03857716]),
 array([ 0.31282536,  2.27098698]),
 array([ 2.46506266, -0.70171807]),
 array([ 1.60178798,  0.07056172])]

### The Gaussian HMM class

In [70]:
class hmmg(object):
    def __init__(self):#(self, A, B, pi):
#         self.A = A
#         self.B = B
#         self.pi = pi
#         self.num_state = A.shape[0]
        pass
        
    def forward_pass(self, obs):
        alpha = np.zeros((len(obs), self.num_state))
        c1 = 1. / sum(self.pi * self.B[obs[0],:])
        alpha[0,:] = c1 * (self.pi * self.B[obs[0],:])
        cs = [c1]
        
        for t in range(1, len(obs)):
            ct = 1. / sum(self.A.dot(alpha[t-1,:]) * B[obs[t],:])
            alpha[t,:] = ct * (self.A.dot(alpha[t-1,:]) * B[obs[t],:])
            cs.append(ct)
    
        return alpha, np.array(cs)
    
    def backward_pass(self, obs):
        a, c = self.forward_pass(obs)
        T = len(obs) - 1
        beta = np.zeros((len(obs), self.num_state))
        beta[T,:] = np.array([c[T] for _ in range(self.num_state)])
        for t in range(T-1, -1, -1):
            beta[t,:] = c[t] * A.T.dot(B[obs[t+1],:] * beta[t+1])
            
        return beta
    
    def _delta(self, obs, alpha, beta):
        T = len(obs)
        delta = np.zeros((T - 1, self.num_state, self.num_state))
        gamma = np.zeros((len(obs), self.num_state))
        
        for t in range(T-1):
            denom = 0
            for k in range(self.num_state):
                for l in range(self.num_state):
                    denom += alpha[t,k] * self.A[l,k] * self.B[obs[t+1],l] * beta[t+1,l]
            
            for i in range(self.num_state):
                for j in range(self.num_state):
                    delta[t,i,j] = (alpha[t,i]*self.A[j,i]*self.B[obs[t+1],j]*beta[t+1,j]) / denom
                gamma[t,i] = sum(delta[t,i,:])
        
        gamma[T-1,:] = alpha[T-1,:] * beta[T-1,:] / sum(alpha[T-1,:] * beta[T-1,:])
                    
        return delta, gamma
            
        
    def _estimate(self, obs, delta, gamma):
        """
        Estimate better parameter values.
        Parameters
        ----------
        obs : ndarray of shape (T,)
        The observation sequence
        delta : ndarray of shape (T-1,N,N)
        The delta probability array
        gamma : ndarray of shape (T,N)
        The gamma probability array
        """
        self.pi = gamma[0,:]
    
        # update A
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]):
                num = 0
                den = 0
                for t in range(len(obs) - 1):
                    num += delta[t,j,i]
                    den += gamma[t,j]
                A[i,j] = num / den
    
        #update B
        for i in range(self.B.shape[0]):
            for j in range(self.B.shape[1]):
                self.B[i,j] = np.sum(gamma[:,j]*(np.array(obs)==i)) / np.sum(gamma[:,j])
                
        # print self.pi, self.A, self.B

    def fit(self, obs, A, B, pi, max_iter=200, tol=1e-3):
        
        M = len(set(obs))
        N = A.shape[0]
        self.num_state = N
        self.A = A
        self.B = B
        self.pi = pi
        
        p = np.inf
        for i in range(max_iter):
#             print i
            alpha, cs = self.forward_pass(obs)
            P = -(np.log(cs)).sum()
            if (abs(P - p) < tol): break
#             if P < -21550: break
                
            p = P
            beta = self.backward_pass(obs)
            delta, gamma = self._delta(obs, alpha, beta)
            self._estimate(obs, delta, gamma)
            
        
        
    def dp(self,obs):
        ### update for transposed A, B
        delta = np.zeros((len(obs), self.num_state))
        delta[0,:] = self.pi * self.B[:, obs[0]]

        best_path = []
        
        for t in range(1, len(obs)):
            probs = delta[t-1, :].reshape(2, 1) * self.A * self.B[:,obs[t]]

            delta[t,:] = np.max(probs, axis=0)
            i = np.argmax(delta[t,:])
            best = np.argmax(probs[:,i])

            best_path.append(best)

        best_path.append(np.argmax(delta[-1,:]))
        return best_path, np.max(delta[-1,:])
        